# Road

- Road Length
- Road Type ('Class')
    - Local (y/n)
    - Major Arterial (y/n)
    - Umimproved (y/n)
    - Ramp (y/n)
    - Freeway (y/n)
    - Other (y/n)

In [1]:
# package imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely

In [2]:
neighborhoods = gpd.read_file('../../data/raw/Neighborhood_Statistical_Areas/Neighborhood_Statistical_Areas.shp')
neighborhoods = neighborhoods.to_crs(epsg=4326)
neighborhoods = neighborhoods[['OBJECTID', 'GNOCDC_LAB', 'geometry']]
neighborhoods.columns = ['nhood_id', 'nhood', 'geometry']

In [3]:
streets = gpd.read_file('../../data/raw/Road_Centerline/geo_export_c02761a8-1d85-477e-a5cf-01b9f22f4d88.shp')

streets_columns = ['joinid',
                   'fromright',
                   'centerline',
                   'roadclass',
                   'fullname',
                   'toleft',
                   'toright',
                   'fullnameab',
                   'objectid',
                   'shape_leng',
                   'fromleft',
                   'geometry']

streets = streets[streets_columns]

In [4]:
streets_reference = streets[['joinid', 'roadclass', 'fullname', 'fullnameab', 'objectid', 'shape_leng', 'geometry']]

In [5]:
streets_join_nhoods = gpd.sjoin(streets, neighborhoods)

In [6]:
join_many = streets_join_nhoods[streets_join_nhoods.objectid.duplicated(keep=False)].sort_values('objectid')
join_single = streets_join_nhoods[~streets_join_nhoods.objectid.duplicated(keep=False)].sort_values('objectid')

In [7]:
join_many.index = join_many.objectid
join_many.drop_duplicates('objectid', inplace=True)

centroids = join_many.geometry.centroid
centroids = gpd.GeoDataFrame(centroids)
centroids.columns = ['geometry']

centroids_nhoods = gpd.sjoin(centroids, neighborhoods, how='left')
centroids_nhoods['objectid'] = centroids_nhoods.index
centroids_nhoods = centroids_nhoods.merge(streets_reference, on='objectid')

subset = ['joinid',
          'fullname',
          'fullnameab',
          'objectid',
          'shape_leng',
          'geometry_y',
          'nhood_id',
          'nhood']

centroids_nhoods = centroids_nhoods[subset]

column_names = ['joinid',
                'fullname',
                'fullnameab',
                'objectid',
                'shape_leng',
                'geometry',
                'nhood_id',
                'nhood']

centroids_nhoods.columns = column_names

In [8]:
subset = ['joinid', 'fullname', 'fullnameab', 'objectid', 'shape_leng', 'geometry', 'nhood_id', 'nhood']
streets_nhoods = join_single[subset]

In [9]:
streets_nhoods = pd.concat([streets_nhoods, centroids_nhoods], ignore_index=True)

In [10]:
subset = ['joinid', 'shape_leng', 'nhood_id', 'geometry']

dissolve_by = ['joinid', 'nhood_id']

streets_dissolved = streets_nhoods[subset].dissolve(by=dissolve_by, aggfunc='sum')

streets_dissolved = streets_dissolved.reset_index()

In [11]:
ref_columns = ['joinid', 'roadclass', 'fullname', 'fullnameab']
street_mode = lambda x:x.value_counts().index[0]
street_info = streets_reference[ref_columns].groupby('joinid').agg(street_mode).reset_index()

In [12]:
streets_by_neighborhood = streets_dissolved.merge(street_info, on='joinid')
streets_by_neighborhood = streets_by_neighborhood[~streets_by_neighborhood[['joinid', 'nhood_id']].duplicated()]
streets_by_neighborhood['roadclass'] = streets_by_neighborhood.roadclass.fillna('Other')

streets_by_neighborhood['segment_id'] = streets_by_neighborhood.nhood_id.astype('str') \
                                        + '_' \
                                        + streets_by_neighborhood.joinid.astype('str')

In [13]:
streets_by_neighborhood = pd.get_dummies(streets_by_neighborhood, prefix='class', columns=['roadclass'])

In [14]:
streets_by_neighborhood.to_pickle('../../data/interim/features/streets_by_nhood.pickle')